In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import datetime
#from implementations import *
#from split_data import * 
%load_ext autoreload
%autoreload 2


In [2]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd

In [3]:
# Main external library : Natural Language Toolkit (nltk)
import nltk
wn = nltk.WordNetLemmatizer()
ps = nltk.PorterStemmer()

nltk.download('punkt')
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/teframartin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Load tweets

In [4]:
DIRECTORY = "data/"

# Note: it seems that the data is already to lower case, so no need to apply lower() to the text
pos_data = pd.read_fwf("data/train_pos.txt", header=None, names=["text"]).drop_duplicates().apply(lambda x: x.str.lower())
pos_data["labels"] = 1
neg_data = pd.read_fwf('data/train_neg.txt', header=None, names=["text"]).drop_duplicates().apply(lambda x: x.str.lower())
neg_data["labels"] = 0

### Cleaning Data

In [5]:
from collections import Counter
import re
import string
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
pd.set_option('display.max_colwidth',100)

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/teframartin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/teframartin/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/teframartin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
# A word that is so common that there is no need to use it in a search
ENGLISH_STOP_WORDS = nltk.corpus.stopwords.words('english')

# Adding few extra stop word
ENGLISH_STOP_WORDS = ENGLISH_STOP_WORDS + ['im', 'dont','dunno', 'cant', ' 2 ', "'s", ' u ', ' x ']

In [7]:
#Concatenate the two training sets of positive and negative tweets
train_data = pd.concat([pos_data, neg_data], ignore_index=True)

In [8]:
# Calculate the most common words used in the set of all tweets
def get_most_common_words(txt,limit):
    return Counter(txt.split()).most_common()[:limit]

# Remove from tweets the punctuation and stop words (= a word that is so common that there is no need to use it in a search.)
def clean_tweet(tweet):
    tweet = "".join([w for w in tweet if w not in string.punctuation])
    tokens = re.split('\W+', tweet)
    tweet = [word for word in tokens if word not in ENGLISH_STOP_WORDS]
    return tweet

# Change any word belonging to the same word-family into a common word (changing/changes/changed.. ==> change)
def lemmatization(token_tweet):
    tweet = [wn.lemmatize(word) for word in token_tweet]
    return tweet

# Concatenate the tokennized tweet into a all text like at the beginning
def concatenate(lst):
    concatenate_tweet = ''
    for elem in lst:
        concatenate_tweet = concatenate_tweet + ' ' + elem
    return concatenate_tweet

In [9]:
# Apply the clean_tweet transformation
train_data['text'] = train_data['text'].apply(lambda x : clean_tweet(x)).apply(lambda x : lemmatization(x))

### Creation of the model

1. Initialize a task-specific model

In [10]:
model = ClassificationModel("roberta", "roberta-base", use_cuda=False)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

2. Train the model with train_model()

In [16]:
model.train_model(train_data.sample(frac=0.005), output_dir="outputs/roberta", args={"overwrite_output_dir": True, "num_train_epochs": 1})

  0%|          | 0/907 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/114 [00:00<?, ?it/s]

(114, 0.6658787920809629)

3. Evaluate the model

In [18]:
# sample an other part of the train full dataset and measure the accuracy
# with eval_model()
result, model_outputs, wrong_predictions = model.eval_model(train_data.sample(frac=0.005))

  0%|          | 0/907 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Running Evaluation:   0%|          | 0/114 [00:00<?, ?it/s]

In [20]:
result, model_outputs

({'mcc': 0.2992819375325854,
  'tp': 278,
  'tn': 311,
  'fp': 174,
  'fn': 144,
  'auroc': 0.7453754824839987,
  'auprc': 0.6712635677654164,
  'eval_loss': 0.5819035491399598},
 array([[ 0.069982  , -0.18846767],
        [-0.33906487,  0.22555636],
        [-0.05236403, -0.10697252],
        ...,
        [-0.33091792,  0.27910367],
        [ 0.16295424, -0.31754607],
        [-0.39224237,  0.28036678]]))

4. Make predictions on (unlabelled) data

In [ ]:
# create the submission file on the test dataset
#  with predict()